# XML은 HTML 처럼 태그 기반으로 자료를 저장한 포멧
* xml parser을 통해서 str을 xml로 변환하는 작업이 필요
* xml로 변환이 되면 태그 기반으로 자료를 찾아서 정리
* 태그에서 자료를 추출할 때는 beautifulsoup이라는 라이브러리를 이용

# 경기도 아파트 정보 xml로 추출하기

In [4]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import re
load_dotenv("./naver_search_web/data/")

False

In [5]:
url="https://openapi.gg.go.kr/TBGGDLNGPRCAPTTRDSRCM"
payload = dict(KEY=os.getenv("gg_api"), Type="xml", pIndex=1, pSize=10)
r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
response = r.text
response

https://openapi.gg.go.kr/TBGGDLNGPRCAPTTRDSRCM?Type=xml&pIndex=1&pSize=10
200


'<?xml version="1.0" encoding="UTF-8"?>\n\n<TBGGDLNGPRCAPTTRDSRCM>\n  <head>\n    <list_total_count>134842</list_total_count>\n    <RESULT>\n      <CODE>INFO-000</CODE>\n      <MESSAGE>정상 처리되었습니다.</MESSAGE>\n    </RESULT>\n    <api_version>1.0v</api_version>\n  </head>\n  <row>\n    <NO>134904</NO>\n    <REFINE_STATE_CD></REFINE_STATE_CD>\n    <LOTNO_ADDR>996</LOTNO_ADDR>\n    <LOTNO_ORIGNO>996</LOTNO_ORIGNO>\n    <LOTNO_VICENO>0</LOTNO_VICENO>\n    <HSCOMPLEX_NM>주공그린빌</HSCOMPLEX_NM>\n    <PRVTUSE_AR>84.8</PRVTUSE_AR>\n    <CONTRACT_YM>2018-01</CONTRACT_YM>\n    <CONTRACT_DAY>1~10</CONTRACT_DAY>\n    <DELNG_AMT>26900</DELNG_AMT>\n    <FLOOR_NM>5</FLOOR_NM>\n    <BUILD_YY>2003</BUILD_YY>\n    <ROAD_NM>통일로802번길</ROAD_NM>\n    <REFINE_CMPLT_YN></REFINE_CMPLT_YN>\n    <CREAT_DE></CREAT_DE>\n    <REFINE_LOTNO_ADDR></REFINE_LOTNO_ADDR>\n    <REFINE_ROADNM_ADDR></REFINE_ROADNM_ADDR>\n    <REFINE_ZIPNO></REFINE_ZIPNO>\n    <REFINE_WGS84_LOGT></REFINE_WGS84_LOGT>\n    <REFINE_WGS84_LAT></REFINE

In [ ]:
print(type(response))

* xml로 데이터를 받으면 처음에는 단순 문자열로 받게 됨
* beautifulsoup을 이용해 xml로 변환후 select, select_one을 이용해 css 셀렉터 기반으로 데이터가 있는 위치를 찾아서 내용 추출
* select: 해당 태그나 css를 문서에서 모두 찾아서 list로 반환
* select_one: 해당 태그나 css를 문서에서 찾아서 가장 앞의 것 1개만 반환
* find: 해당 태그를 찾아서 가장 앞의 것 1개만 반환
* find_all: 해당 태그를 문서에서 모두 찾아서 list로 반환
* 찾아온 태그.name: 태그 이름 반환
* 찾아온 태그.text, .string: 태그 안쪽의 텍스트 반환

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
soup = bs(response, 'xml')
soup

In [ ]:
soup.select_one("list_total_count").name

In [ ]:
soup.select_one("list_total_count").text

In [ ]:
soup.select_one("list_total_count").string

In [ ]:
soup.select("row")

In [ ]:
aparts = soup.select("row")

In [ ]:
for apart in aparts[:2]:
#    print(apart)
    print(apart.select("NO"))
    print(type(apart.select("NO")))
    print(type(apart.select_one("NO")))
    print(apart.select_one("NO").text)

In [ ]:
aparts[0].name

In [6]:
cols = ['NO', 'LOTNO_ADDR', 'LOTNO_ORIGNO', 'LOTNO_VICENO',
       'HSCOMPLEX_NM', 'PRVTUSE_AR', 'CONTRACT_YM', 'CONTRACT_DAY',
       'DELNG_AMT', 'FLOOR_NM', 'BUILD_YY', 'ROAD_NM', 'ADDR']

In [ ]:
key : []
key2: []

In [ ]:
aparts[0].select('NO')

In [9]:
result = {}
for key in cols:
    temp = []
    for value in soup.select(key):
       # print(value.text)
        temp.append(value.text)
    result[key] = temp    

result
df = pd.DataFrame(result)
df

,NO,LOTNO_ADDR,LOTNO_ORIGNO,LOTNO_VICENO,HSCOMPLEX_NM,PRVTUSE_AR,CONTRACT_YM,CONTRACT_DAY,DELNG_AMT,FLOOR_NM,BUILD_YY,ROAD_NM,ADDR
0,134904,996,996,0,주공그린빌,84.8,2018-01,1~10,26900,5,2003,통일로802번길,경기도 고양덕양구 관산동
1,134905,996,996,0,주공그린빌,75.95,2018-01,11~20,27000,10,2003,통일로802번길,경기도 고양덕양구 관산동
2,134906,996,996,0,주공그린빌,84.51,2018-01,11~20,25500,2,2003,통일로802번길,경기도 고양덕양구 관산동
3,134907,996,996,0,주공그린빌,59.92,2018-01,11~20,23700,10,2003,통일로802번길,경기도 고양덕양구 관산동
4,134908,996,996,0,주공그린빌,84.8,2018-01,11~20,27450,4,2003,통일로802번길,경기도 고양덕양구 관산동


In [10]:
result = {}
for key in cols:
    for value in soup.select(key):
        result.setdefault(key, []).append(value.text)

df = pd.DataFrame(result)
df

,NO,LOTNO_ADDR,LOTNO_ORIGNO,LOTNO_VICENO,HSCOMPLEX_NM,PRVTUSE_AR,CONTRACT_YM,CONTRACT_DAY,DELNG_AMT,FLOOR_NM,BUILD_YY,ROAD_NM,ADDR
0,134904,996,996,0,주공그린빌,84.8,2018-01,1~10,26900,5,2003,통일로802번길,경기도 고양덕양구 관산동
1,134905,996,996,0,주공그린빌,75.95,2018-01,11~20,27000,10,2003,통일로802번길,경기도 고양덕양구 관산동
2,134906,996,996,0,주공그린빌,84.51,2018-01,11~20,25500,2,2003,통일로802번길,경기도 고양덕양구 관산동
3,134907,996,996,0,주공그린빌,59.92,2018-01,11~20,23700,10,2003,통일로802번길,경기도 고양덕양구 관산동
4,134908,996,996,0,주공그린빌,84.8,2018-01,11~20,27450,4,2003,통일로802번길,경기도 고양덕양구 관산동


# 전체 데이터 수집해서 csv 파일로 저장하기

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
import time
import os
import re
load_dotenv("./naver_search_web/data/")

False

In [8]:
page_num = 1
total_page = 1
result = {}

while page_num <= total_page:
    url="https://openapi.gg.go.kr/TBGGDLNGPRCAPTTRDSRCM"
    payload = dict(KEY=os.getenv("gg_api"), Type="xml", pIndex=page_num, pSize=1000)
    r = requests.get(url, params=payload)
    #print(r.url)
    #print(r.status_code)
    response = r.text
    soup = bs(response, 'xml')

    total_page=int(soup.select_one("list_total_count").text) // 1000 +1

    result = {}
    for key in cols:
        for value in soup.select(key):
            result.setdefault(key, []).append(value.text)
    print(f"{page_num}/{total_page}수집중입니다", end="\r")
    page_num += 1
    
    time.sleep(0.2)

df = pd.DataFrame(result)
df

,NO,LOTNO_ADDR,LOTNO_ORIGNO,LOTNO_VICENO,HSCOMPLEX_NM,PRVTUSE_AR,CONTRACT_YM,CONTRACT_DAY,DELNG_AMT,FLOOR_NM,BUILD_YY,ROAD_NM,ADDR
0,134904,996,996,0,주공그린빌,84.8,2018-01,1~10,26900,5,2003,통일로802번길,경기도 고양덕양구 관산동
1,134905,996,996,0,주공그린빌,75.95,2018-01,11~20,27000,10,2003,통일로802번길,경기도 고양덕양구 관산동
2,134906,996,996,0,주공그린빌,84.51,2018-01,11~20,25500,2,2003,통일로802번길,경기도 고양덕양구 관산동
3,134907,996,996,0,주공그린빌,59.92,2018-01,11~20,23700,10,2003,통일로802번길,경기도 고양덕양구 관산동
4,134908,996,996,0,주공그린빌,84.8,2018-01,11~20,27450,4,2003,통일로802번길,경기도 고양덕양구 관산동
